In [1]:
import pandas as pd

In [19]:
#Reading latest dataframe

#df = pd.read_csv('Dec1_trending_video.csv')

In [20]:
category_list = pd.read_csv('category_list.csv')

In [21]:
category_list

id          snippet.title
0    1       Film & Animation
1    2       Autos & Vehicles
2   10                  Music
3   15         Pets & Animals
4   17                 Sports
5   18           Short Movies
6   19        Travel & Events
7   20                 Gaming
8   21          Videoblogging
9   22         People & Blogs
10  23                 Comedy
11  24          Entertainment
12  25        News & Politics
13  26          Howto & Style
14  27              Education
15  28   Science & Technology
16  29  Nonprofits & Activism
17  30                 Movies
18  31        Anime/Animation
19  32       Action/Adventure
20  33               Classics
21  34                 Comedy
22  35            Documentary
23  36                  Drama
24  37                 Family
25  38                Foreign
26  39                 Horror
27  40         Sci-Fi/Fantasy
28  41               Thriller
29  42                 Shorts
30  43                  Shows
31  44               Trailers

In [23]:
import os

from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors
import google_auth_oauthlib.flow

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

flow = InstalledAppFlow.from_client_secrets_file(
 "Youtube_Oauth.json", scopes = scopes)

flow.run_local_server(port = 8080)

credentials = flow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725695838850-3pp8b4qjgqebrq8iqdag2pmkhcmp7h3i.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=CEGZYK6FtVKnvLHahWYoroLTXIuJxO&access_type=offline


In [25]:
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "Youtube_Oauth.json"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, credentials=credentials)

In [26]:
request = youtube.videos().list(
    part="snippet,contentDetails,statistics",
    chart="mostPopular",
    regionCode="US"
)
response = request.execute()

print(response)

{'kind': 'youtube#videoListResponse', 'etag': 'oN1Ailjvd6fZkH4qLmCD4RFnm5w', 'items': [{'kind': 'youtube#video', 'etag': '5VQ7bqd9rKyTCUKRWsrATsZRRBE', 'id': 'TNl7ZcnDB0Q', 'snippet': {'publishedAt': '2022-11-29T21:27:06Z', 'channelId': 'UCooTLkxcpnTNx6vfOovfBFA', 'title': 'Iran vs. United States Highlights | 2022 FIFA World Cup', 'description': 'The United States and Iran faced off in a must-win situation for the US. The US came out aggressive and got off multiple shots on goal but they did not score. That all changed in the 38th minute when Sergiño Dest found Christian Pulisic with a perfectly timed pass and Pulisic put it into the back of the net. Pulisic was visibly hurt on the play. He wouldn’t return in the second half. Josh Sargent would come out of the game later after appearing to injure his leg. Iran had a shot on goal late on a free kick, but the US would hold on, marking three-straight knockout stage appearances in their last three World Cup appearances.\n\n#FOXSoccer #FIFA

# Trending Video Daily Update Function

In [27]:
# A function update trending video daily
#Parameter:
    #df: current dataset
    #response: the first page of trending video retrieve by API.
    #No out put value, the df will be directly updated.
    #Don't forget to output the dataframe to csv

def Trending_Daily_Update(df,response):
    for i in range(len(response['items'])):
        try:
            likeCount = response['items'][i]['statistics']['likeCount']
        except:
            likeCount = None
        
        try:
            commentCount = response['items'][i]['statistics']['commentCount']
        except:
            commentCount = None
        
        try:
            favoriteCount = response['items'][i]['statistics']['favoriteCount']
        except:
            favoriteCount = None
            
        try:
            tags = response['items'][i]['snippet']['tags']
        except:
            tags = None
        
        df.loc[len(df.index)] = [response['items'][i]['id'],
                          response['items'][i]['snippet']['publishedAt'],
                          response['items'][i]['snippet']['title'],
                          response['items'][i]['snippet']['description'],
                          response['items'][i]['snippet']['channelId'],
                           response['items'][i]['snippet']['channelTitle'],
                           tags,
                           response['items'][i]['contentDetails']['dimension'],
                           response['items'][i]['contentDetails']['definition'],
                           response['items'][i]['contentDetails']['caption'],
                           response['items'][i]['contentDetails']['licensedContent'],
                           response['items'][i]['statistics']['viewCount'],
                           likeCount,
                           commentCount,
                           favoriteCount,
                           response['items'][0]['snippet']['categoryId'],
                           category_list[category_list['id'] == int(response['items'][0]['snippet']['categoryId'])].iloc[0,1]
                            
                                ]
    if 'nextPageToken' in response.keys():
        nextPageToken = response['nextPageToken']
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            chart="mostPopular",
            regionCode="US",
            pageToken = response['nextPageToken']
            )
        
        response = request.execute()
        Trending_Daily_Update(df,response)
        
    else:
        None

In [28]:
Trending_Daily_Update(df,response)

In [29]:
df

id          publish_time  \
0     359IGLUlNhE  2022-11-24T15:49:57Z   
1     ml9eVc6ZR5A  2022-11-24T19:43:29Z   
2     kFGWYk1khMg  2022-11-25T02:01:13Z   
3     359IGLUlNhE  2022-11-24T15:49:57Z   
4     ml9eVc6ZR5A  2022-11-24T19:43:29Z   
...           ...                   ...   
998   x9NMPnI91Hg  2022-11-22T11:00:28Z   
999   skl6N3zGv-s  2022-11-24T16:00:08Z   
1000  JHXDWBWSz00  2022-11-25T13:00:13Z   
1001  k-jjWq0TFMc  2022-11-25T23:00:35Z   
1002  RuDsBrSczis  2022-11-25T05:30:10Z   

                                                  title  \
0          Watch: Annual Macy's Thanksgiving Day parade   
1                                   What Happened To Me   
2                                Christmas With My Son!   
3          Watch: Annual Macy's Thanksgiving Day parade   
4                                   What Happened To Me   
...                                                 ...   
998      Quick Share the holidays with Galaxy | Samsung   
999                  Rammstein - Adieu (Official Video)   
1000                    Who Killed Blue Mod in Among Us   
1001    Goat Simulator 3 is the best new game available   
1002  WYTB (Full Video) Karan Aujla ft Gurlej Akhtar...   

                                            description  \
0     In 1924, Macy's held its first parade but inst...   
1                  Why I Left YouTube....and am I back?   
2     Ho ho ho\n\nMerch: https://baylenlevine.com/\n...   
3     In 1924, Macy's held its first parade but inst...   
4                  Why I Left YouTube....and am I back?   
...                                                 ...   
998   Sharing is a kind of magic. No matter where we...   
999   Order the single: https://rammstein.lnk.to/Adi...   
1000  Today @Zud is joined with @SSundee, @Sigils an...   
1001  Goat Simulator 3 is the game I never knew I wa...   
1002  Listen on\n\nApple Music - https://music.apple...   

                     channelId        channelTitle  \
0     UCP6HGa63sBC7-KHtkme-p-g           USA TODAY   
1     UC8rVYYeV6ekdAYIAYlaeTVA        LostNUnbound   
2     UCwsjrSWqxLdcZrd9xVpmxog       Baylen Levine   
3     UCP6HGa63sBC7-KHtkme-p-g           USA TODAY   
4     UC8rVYYeV6ekdAYIAYlaeTVA        LostNUnbound   
...                        ...                 ...   
998   UCWwgaK7x0_FR1goeSRazfsQ             Samsung   
999   UCYp3rk70ACGXQ4gFAiMr1SQ  Rammstein Official   
1000  UCgYJLAsnjCZk9ocyIywgbiQ                 Zud   
1001  UCzXwjTI6c6mVn6oui_p6oiw              SMii7Y   
1002  UCm9SZAl03Rev9sFwloCdz1g      Rehaan Records   

                                                   tags dimension definition  \
0     ['usa today', 'usa today news', 'macys', 'para...        2d         hd   
1     ['LostNUnbound', 'LNU', 'What happened to LNU'...        2d         hd   
2     ['baylen levine', 'baylen levine vlogs', 'beni...        2d         hd   
3     ['usa today', 'usa today news', 'macys', 'para...        2d         hd   
4     ['LostNUnbound', 'LNU', 'What happened to LNU'...        2d         hd   
...                                                 ...       ...        ...   
998                                           [Samsung]        2d         hd   
999   [Rammstein, Rammstein Official, Official, Offi...        2d         hd   
1000  [Among Us, Imposter, Among Us Mod, Zud, Among ...        2d         hd   
1001  [goat simulator 3, goat simulator 3 gameplay, ...        2d         hd   
1002  [karan aujla wytb, karan aujla new punjabi son...        2d         hd   

     caption  licensedContent viewCount likeCount commentCount favoriteCount  \
0      False             True   1606606    9721.0        778.0             0   
1      False             True    480725   58118.0       7686.0             0   
2      False             True    513649   33185.0       2720.0             0   
3      False             True   1606606    9721.0        778.0             0   
4      False             True    480725   58118.0       7686.0        

In [30]:
#File Name Formula: Date_tending_video.csv
#Example:Dec1_trending_video.csv

#df.to_csv('')